# 크롤링/스크레이핑 실전과 데이터 활용 

- 데이터 세트 추출과 활용
- API로 데이터 수집하고 활용
- 시계열 데이터 수집하고 활용
- 열린 데이터 수집과 활용
- 웹 페이지 자동 조작
- 자바스크립트를 이용한 페이지 스크레이핑
- 추출한 데이터 활용

In [1]:
import os
from os.path import exists

def make_dir(dir_name):
    ''' 디렉토리 생성 '''
    
    if not exists(dir_name):
        os.mkdir(dir_name)
        msg = "{} 디렉토리를 생성하였습니다!".format(dir_name)
    else:
        msg = "{} 디렉토리가 이미 존재합니다!".format(dir_name)
        
    return msg

## 데이터 세트 추출과 활용 
- Index of /kowiki/

- 위키백과 데이터 세트 다운로드
- 위키백과 데이터 세트에서 문장 추출

In [3]:
make_dir('data/dataset')

'data/dataset 디렉토리를 생성하였습니다!'

In [4]:
# ! wget https://dumps.wikimedia.org/kowiki/20181101/ -q -O -

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Strict//EN"
        "http://www.w3.org/TR/xhtml1/DTD/xhtml1-strict.dtd">

<html xmlns="http://www.w3.org/1999/xhtml" xml:lang="en" lang="en">
<head>
        <meta http-equiv="Content-Type" content="text/html; charset=utf-8"/>
        <title>kowiki dump progress on 20181101</title>
        <link rel="stylesheet" type="text/css" href="/dumps.css" />
        <style type="text/css">
                .siteinfo {
                        text-align: center;
                }
                li {
                        list-style-type: none;
                        padding: 0.5em 1.5em 0.5em 1.5em;
                        background: #fff;
                        margin-bottom: 1em;
                }
                li li {
                        background-color: white;
                        box-shadow: none;
                        border-top: none;
                        padding: 0px;
                        margin-bottom: 0em;
              

In [5]:
#! wget https://dumps.wikimedia.org/kowiki/20181101/ -O ./data/dataset/kowiki-2018110-index.html

--2018-11-15 11:15:10--  https://dumps.wikimedia.org/kowiki/20181101/
Resolving dumps.wikimedia.org (dumps.wikimedia.org)... 208.80.154.7
Connecting to dumps.wikimedia.org (dumps.wikimedia.org)|208.80.154.7|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15049 (15K) [text/html]
Saving to: './data/dataset/kowiki-2018110-index.html'

     0K .......... ....                                       100% 2.10M=0.007s

2018-11-15 11:15:11 (2.10 MB/s) - './data/dataset/kowiki-2018110-index.html' saved [15049/15049]



In [6]:
from bs4 import BeautifulSoup
import pandas as pd

filepath = './data/dataset/kowiki-2018110-index.html'
with open(filepath, encoding='utf-8') as fp:
    soup = BeautifulSoup(fp, 'lxml')

soup.title

<title>kowiki dump progress on 20181101</title>

In [7]:
done_tags = soup.find_all('li', 'done')
len(done_tags)

31

In [8]:
done_tag = done_tags[0]
done_tag

<li class="done"><span class="updates">2018-11-02 16:02:35</span> <span class="status">done</span> <span class="title">Articles, templates, media/file descriptions, and primary meta-pages, in multiple bz2 streams, 100 pages per stream</span><ul><li class="file"><a href="/kowiki/20181101/kowiki-20181101-pages-articles-multistream.xml.bz2">kowiki-20181101-pages-articles-multistream.xml.bz2</a> 603.6 MB</li>
<li class="file"><a href="/kowiki/20181101/kowiki-20181101-pages-articles-multistream-index.txt.bz2">kowiki-20181101-pages-articles-multistream-index.txt.bz2</a> 10.5 MB</li></ul></li>

In [9]:
updates = done_tag.find('span', 'updates').get_text()
updates

'2018-11-02 16:02:35'

In [10]:
title = done_tag.find('span', 'title').get_text()
title

'Articles, templates, media/file descriptions, and primary meta-pages, in multiple bz2 streams, 100 pages per stream'

In [11]:
file_tags = done_tag.find_all('li', 'file')
files = []
for file_tag in file_tags:
    file_dict = {}
    file_name = file_tag.find('a').get_text()
    file_link = file_tag.find('a').get('href')
    
    file_dict['FileName'] = file_name
    file_dict['FileLink'] = file_link
    files.append(file_dict)

files

[{'FileName': 'kowiki-20181101-pages-articles-multistream.xml.bz2',
  'FileLink': '/kowiki/20181101/kowiki-20181101-pages-articles-multistream.xml.bz2'},
 {'FileName': 'kowiki-20181101-pages-articles-multistream-index.txt.bz2',
  'FileLink': '/kowiki/20181101/kowiki-20181101-pages-articles-multistream-index.txt.bz2'}]

In [12]:
from bs4 import BeautifulSoup
import pandas as pd

def getKowikiInfos(filepath):

    with open(filepath, encoding='utf-8') as fp:
        soup = BeautifulSoup(fp, 'lxml')

    Updates  = []
    Title    = []
    FileList = []   # 파일이름 리스트
    Files    = []   # 파일상세 리스트

    done_tags = soup.find_all('li', 'done')

    for done_tag in done_tags:
        # 등록날짜
        updates = done_tag.find('span', 'updates').get_text()

        # 제목
        title = done_tag.find('span', 'title').get_text()

        # 파일리스트
        file_tags = done_tag.find_all('li', 'file')
        files = []
        filenames = []
        for file_tag in file_tags:
            file_dict = {}
            file_name = file_tag.find('a').get_text()
            file_link = file_tag.find('a').get('href')

            file_dict['FileName'] = file_name
            file_dict['FileLink'] = file_link
            files.append(file_dict)

            filenames.append(file_name)

        Updates.append(updates)
        Title.append(title)
        FileList.append(tuple(filenames))
        Files.append(files)

    kokiwi_df = pd.DataFrame({ 'Updates'  : Updates, 
                               'Title'    : Title,
                               'FileList' : FileList,
                               'FileInfo' : Files }, 
                               columns = ['Updates', 'Title', 'FileList', 'FileInfo'])

    return kokiwi_df

In [13]:
filepath = './data/dataset/kowiki-2018110-index.html'
kokiwi_df = getKowikiInfos(filepath)

In [14]:
kokiwi_df.head()

,Updates,Title,FileList,FileInfo
0,2018-11-02 16:02:35,"Articles, templates, media/file descriptions, ...",(kowiki-20181101-pages-articles-multistream.xm...,[{'FileName': 'kowiki-20181101-pages-articles-...
1,2018-11-06 05:30:17,All pages with complete edit history (.7z),"(kowiki-20181101-pages-meta-history.xml.7z,)",[{'FileName': 'kowiki-20181101-pages-meta-hist...
2,2018-11-06 00:03:51,All pages with complete page edit history (.bz2),"(kowiki-20181101-pages-meta-history.xml.bz2,)",[{'FileName': 'kowiki-20181101-pages-meta-hist...
3,2018-11-03 21:35:06,Log events to all pages and users.,"(kowiki-20181101-pages-logging.xml.gz,)",[{'FileName': 'kowiki-20181101-pages-logging.x...
4,2018-11-03 00:39:38,"All pages, current versions only.","(kowiki-20181101-pages-meta-current.xml.bz2,)",[{'FileName': 'kowiki-20181101-pages-meta-curr...


In [15]:
kokiwi_df['FileList'][0]

('kowiki-20181101-pages-articles-multistream.xml.bz2',
 'kowiki-20181101-pages-articles-multistream-index.txt.bz2')

## 자연어 처리를 사용한 빈출 단어 추출 
- 형태소 분석 : 자연어 처리의 기본 기술

- 주어진 단어를 형태소라는 최소 단어로 분할해서 품사 등을 판별하는 것을 의미
- 형태소 분석 라이브러리 : KoNLLPy, 파이썬 한국어 NLP
- 한나눔
- 꼬꼬마
- Komoran
- MeCab
- 트위트

### c5-01_konlpy_sample
- KoNLPy로 형태소 분석

In [17]:
from konlpy.tag import Kkma

kkma   = Kkma()
malist = kkma.pos("아버지 가방에 들어가신다.")
malist

[('아버지', 'NNG'),
 ('가방', 'NNG'),
 ('에', 'JKM'),
 ('들어가', 'VV'),
 ('시', 'EPH'),
 ('ㄴ다', 'EFN'),
 ('.', 'SF')]

### 문장에서 빈출 단어 추출 
- c5-02_word_frequency 
- 위키백과 문장에서 빈출 단어 추출

In [19]:
%%writefile  word_frequency.py
# %%writefile  ./modules/word_frequency.py
import sys
import os
from glob import glob
from collections import Counter
from konlpy.tag import Kkma

def main():
    """
    명령라인 매개변수로 지정한
    디렉터리 내부의 파일을 읽어 들이고
    빈출 단어를 출력합니다.
    """
    # 명령어의 첫 번째 매개변수로
    # WikiExtractor의 출력 디렉터리를 지정합니다.
    input_dir = sys.argv[1]
    print('input_dir :', input_dir)
    kkma = Kkma()
    
    # 단어의 빈도를 저장하기 위한 Counter 객체를 생성합니다.
    # Counter 클래스는 dict를 상속받는 클래스입니다.
    frequency = Counter()
    count_proccessed = 0
    
    # glob()으로 와일드카드 매치 파일 목록을 추출하고
    # 매치한 모든 파일을 처리합니다.
    # print('before for :', glob(os.path.join(input_dir, 'kowiki-*')))
    # for path in glob(os.path.join(input_dir, '*', 'wiki_*')):
    for path in glob(os.path.join(input_dir, 'kowiki-*')):
        print('path :', path)
        print('Processing {0}...'.format(path), file=sys.stderr)
        
        # 파일을 엽니다.
        with open(path) as file:
            # 파일 내부의 모든 기사에 반복을 돌립니다.
            for content in iter_docs(file):
                # 페이지에서 명사 리스트를 추출합니다.
                tokens = get_tokens(kkma, content)
                
                # Counter의 update() 메서드로 리스트 등의 반복 가능 객체를 지정하면
                # 리스트에 포함된 값의 출현 빈도를 세어줍니다.
                frequency.update(tokens)
                
                # 10,000개의 글을 읽을 때마다 간단하게 출력합니다.
                count_proccessed += 1
                if count_proccessed % 10000 == 0:
                    print('{0} documents were processed.'
                        .format(count_proccessed),file=sys.stderr)
    
    # 모든 기사의 처리가 끝나면 상위 30개의 단어를 출력합니다
    for token, count in frequency.most_common(30):
        print(token, count)

        
def iter_docs(file):
    """
    파일 객체를 읽어 들이고
    기사의 내용(시작 태그 <doc>와 종료 태그 </doc> 사이의 텍스트)를 꺼내는
    제너레이터 함수
    """
    for line in file:
        if line.startswith('<doc '):
            # 시작 태그가 찾아지면 버퍼를 초기화합니다.
            buffer = []
        elif line.startswith('</doc>'):
            # 종료 태그가 찾아지면 버퍼의 내용을 결합한 뒤 yield합니다.
            content = ''.join(buffer)
            yield content
        else:
            # 시작 태그/종료 태그 이외의 줄은 버퍼에 추가합니다.
            buffer.append(line)

            
def get_tokens(kkma, content):
    """
    문장 내부에 출현한 명사 리스트를 추출하는 함수
    """
    # 명사를 저장할 리스트입니다.
    tokens = []
    node = kkma.pos(content)
    for (taeso, pumsa) in node:
        # 고유 명사와 일반 명사만 추출합니다.
        if pumsa in ('NNG', 'NNP'):
            tokens.append(taeso)
    return tokens

if __name__ == '__main__':
    main()

Writing word_frequency.py


In [20]:
make_dir('./data/result')

'./data/result 디렉토리를 생성하였습니다!'

In [21]:
! python word_frequency.py .\data\dataset

input_dir : .\data\dataset
path : .\data\dataset\kowiki-2018110-index.html


Processing .\data\dataset\kowiki-2018110-index.html...
Traceback (most recent call last):
  File "word_frequency.py", line 88, in <module>
    main()
  File "word_frequency.py", line 36, in main
    for content in iter_docs(file):
  File "word_frequency.py", line 71, in iter_docs
    buffer.append(line)
UnboundLocalError: local variable 'buffer' referenced before assignment


## API로 데이터 수집하고 활용 
- 트위트에서 데이터 수집
- 유튜브에서 데이터 수집

# 트위터에서 데이터 수집 
- 데이터 수집에 사용되는 2가지 API

- REST API : 
- HTTP 요청을 전송하면 HTTP 응답을 반환하는 RESTful 형태로 트윗 또는 사용자 정보를 추출, 
- 호출 횟수 제한이 굉장히 엄격 
- 한 사용자당 15분에 15회만 호출 제한이 있으므로, 대규모 데이터수집에는 좋지 않다. 

- Streaming API : 
- 한번의 요청을 보내면 서버와의 연결(컨넥션)을 유지하고, 
- 새로운 데이터가 추가될 때마다 서버가 데이터를 전송해주는 푸시 형태 
- HTTP 요청을 보냈을때 연결을 확립한 상태로 두고 서버에서 메시지를 계속 전송받는 형태

### 트위터 API 인증 
- OAuth 1.0a 인증 필요 (아래 4가지 모두 필요)

- 애플리케니션 단위 발행 
- Cousumer Key 
- Consuber Secret 

- 사용자 단위 발행 
- Access Token 
- Access Token Secret

#### 트위터 인증 정보를 위해 애플리케이션 등록

- [ 애플리케이션 관리 화면 ] 에 트위터 계정으로 로그인
- "Create New App" 버튼 클릭
- 이름, 설명, 웹사이트 를 입력하고 사용약관 동의하면 애플리케이션이 생성된다.
- "Keys and Access Tokens" 링크 클릭 후, 생성된 4개의 키 값을 저장

In [22]:
# Twitter Application Settings
CONSUMER_KEY        = "Rh9PPrGkDRohh5GualkQRErPu"
CONSUMER_SECRET     = "Emg0dTXJXawpWn4EapiyG2aE5bWHGLreFd6vPy4XZMQmVb7ULp"

# Twitter Your Access Token
ACCESS_TOKEN        = "4477564098-AGJFdwERnNyEqVGHaPyy8yyloiPaCXsPob1q1Zy"
ACCESS_TOKEN_SECRET = "otjvkl1U6POGZMw1SpGQHMHlyEPz9Z1TbhPIecFhCuI6e"

## c5-03_rest_api_with_requests_oauthlib 
- Requests OAuthLib 를 이용한 타임라인 추출

- 인증 정보를 사용해 OAuth1Session 객체를 생성
- API 응답이 JSON 형식의 문자열이므로 response.json()으로 파싱
- status는 트윗(Twitter API에서는 Status)를 나타내는 dict

In [31]:
! pip install oauth

  Running setup.py bdist_wheel for oauth: started
  Running setup.py bdist_wheel for oauth: finished with status 'done'
  Stored in directory: C:\Users\Ente_LJS\AppData\Local\pip\Cache\wheels\f7\a8\ca\272f26e0c3973e23fe6720ca1c98fb2f5630263ab11c90af62
Successfully built oauth


In [36]:
from requests_oauthlib import OAuth1Session
import os

# 환경변수에서 인증 정보를 추출합니다.
# CONSUMER_KEY        = os.environ['CONSUMER_KEY']
# CONSUMER_SECRET     = os.environ['CONSUMER_SECRET']
# ACCESS_TOKEN        = os.environ['ACCESS_TOKEN']
# ACCESS_TOKEN_SECRET = os.environ['ACCESS_TOKEN_SECRET']

CONSUMER_KEY        = "Rh9PPrGkDRohh5GualkQRErPu"
CONSUMER_SECRET     = "Emg0dTXJXawpWn4EapiyG2aE5bWHGLreFd6vPy4XZMQmVb7ULp"
ACCESS_TOKEN        = "4477564098-AGJFdwERnNyEqVGHaPyy8yyloiPaCXsPob1q1Zy"
ACCESS_TOKEN_SECRET = "otjvkl1U6POGZMw1SpGQHMHlyEPz9Z1TbhPIecFhCuI6e"


# 인증 정보를 사용해 OAuth1Session 객체를 생성합니다.
twitter = OAuth1Session(CONSUMER_KEY,
                        client_secret=CONSUMER_SECRET,
                        resource_owner_key=ACCESS_TOKEN,
                        resource_owner_secret=ACCESS_TOKEN_SECRET)

# 사용자의 타임라인을 추출합니다.
response = twitter.get('https://api.twitter.com/1.1/statuses/home_timeline.json')

# API 응답이 JSON 형식의 문자열이므로 response.json()으로 파싱합니다.
# status는 트윗(Twitter API에서는 Status라고 부릅니다)를 나타내는 dict입니다.
for status in response.json():
    # 사용자 이름과 트윗을 출력합니다.
    print('@' + status['user']['screen_name'], status['text'])

@ARneipia 앞으로 1일!! https://t.co/u7NEADETqQ
@ARneipia RT @Pokemon_cojp: 11月17日（土）より和菓子をテーマにしたポケモンマスコットが登場！ 全部で6種類あって、どのポケモンかは、お楽しみだよ！ イーブイ茶房へ、ようこそ！ https://t.co/8YHXb9FOm3 #ポケモン https://t.co/wV…
@ARneipia RT @hobby_magazine: 今年最大のヒットキャラクター!?『#ゲゲゲの鬼太郎』より、好評受注中の「#ねこ娘」製品サンプルを自然光フォト！
https://t.co/04gKxlTTHf

#鬼太郎 https://t.co/d2xmOgKxae
@ARneipia RT @kitaroanime50th: 本日発売‼️
#アニメージュ 12月号では #ゲゲゲの鬼太郎 描き下ろし表紙&amp;ピンナップポスターイラストを使用した缶バッジ6種セットの応募者全員サービスも実施中☝️初グッズ化のアニエス&amp;アデルにも注目です✨ https://t.co/c…
@ARneipia 일도 바쁘고 참..뭐하다.. ㅇ 3 ㅇ...
그냥 내 일이나 잘해야지.
@ARneipia 소문이 돌아도 대응할 가치도 못느끼겠고..이젠 남에게 이건 잘못됬다고 말할 에너지조차 밑바닥이라 그냥 흘러가게 내버려두게된다. 내 목구멍이 급하지...
@ARneipia https://t.co/Suv15VrxBP
@ARneipia RT @DetPikachuMovie: Partner up with a legend. @VancityReynolds is #DetectivePikachu in theaters Summer 2019! https://t.co/QyX0TyUP74
@ARneipia RT @Pokemon_cojp: アニメ「ポケットモンスター サン＆ムーン」で放送中の、前髪の長いイーブイのショートストーリー「イーブイどこいくの？」第4話を、期間限定で公開！ 続きは11月18日（日）の放送でチェックしよう！ #アニポケ https://t.co/VcnoK…
@ARneipia ㅋㅋㅋ 허그또에 출연했닼ㅋㅋ 너무 미화해주

### c5-04_rest_api_with_tweepy 
- tweepy 를 이용한 타임라인 추출

- 연결 확립한 상태로 메시지 계속 전송
- 각 메시지는 줄바꿈 코드 CRLF로 구분
- 메시지는 대부분 트윗을 나타내는 JSON 형식의 문자열
- 트윗 이외의 연결을 유지하기 위한 공백과 메타 정보도 전송된다.

In [37]:
import os
import tweepy

# 환경변수에서 인증 정보를 추출합니다.
# CONSUMER_KEY = os.environ['CONSUMER_KEY']
# CONSUMER_SECRET = os.environ['CONSUMER_SECRET']
# ACCESS_TOKEN = os.environ['ACCESS_TOKEN']
# ACCESS_TOKEN_SECRET = os.environ['ACCESS_TOKEN_SECRET']
CONSUMER_KEY        = "Rh9PPrGkDRohh5GualkQRErPu"
CONSUMER_SECRET     = "Emg0dTXJXawpWn4EapiyG2aE5bWHGLreFd6vPy4XZMQmVb7ULp"
ACCESS_TOKEN        = "4477564098-AGJFdwERnNyEqVGHaPyy8yyloiPaCXsPob1q1Zy"
ACCESS_TOKEN_SECRET = "otjvkl1U6POGZMw1SpGQHMHlyEPz9Z1TbhPIecFhCuI6e"

# 인증 정보를 설정합니다.
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

class MyStreamListener(tweepy.StreamListener):
    """
    Streaming API로 추출한 트윗을 처리하는 클래스입니다.
    """
    def on_status(self, status):
        """
        트윗을 받을 때 호출되는 메서드
        매개변수로 트윗을 나타내는 Status 객체가 전달됩니다.
        """
        print('@' + status.author.screen_name, status.text)

# 인증 정보와 StreamListener를 지정해서 Stream 객체를 추출합니다.
stream = tweepy.Stream(auth, MyStreamListener())

# 공개돼 있는 트윗을 샘플링한 스트림을 받습니다.
# 키워드 매개변수인 languages로 한국어 트윗만 추출합니다
stream.sample(languages=['ko'])
# stream.sample(languages=['en'])

@alsa6865 RT @4CTR1CE5: 아 신이시여,,, ,,, 으악 ,,,,,, ㅠ 난 퀴리 &gt;부인&lt;이 아니야,,, ,,, ㅠㅠㅠ https://t.co/UG035Tyblx
@HARUMIM_MB @KH_Vigenhoo 닮았어요..
@pokn775 RT @mondayBlues_88: 여성기자와 컨택 했다고 합니다 찌라시 기사 클릭❌❌❌❌❌❌ #이수역_폭행사건 https://t.co/YAI5HaduCZ
@MARKINGPAINTER RT @muttz__: #윤화평 쥬륵쥬르륵💧 https://t.co/HfYSgSkWhS
@qn0_b8 RT @KINGOD_GUIN: 술래 황민현 (?)
내게 무뚝뚝하게 굴지마요 견딜수가 없어요 https://t.co/n7wIlXYjGC
@kaonattanicha RT @qqstl001: 낙엽 좋아하는 토끼 너무 힐링된다 https://t.co/OrDyMaGrEz
@Up5Down @doridoriyamyam 이렇게 얘기하니까 급 그리워서 해보고 싶긴하네요ㅎㅎ
@chiakitty211 RT @suga_suga__love: 옥택연 분이랑 친척오빠랑 진짜 사진 찍으심 https://t.co/ZWTeNC8dky
@b0boIAoyJi1jXhX RT @1dhkfdhkf22: 아 군무새새끼들아!!!!!!!!! https://t.co/sHtpaCkaQe
@mint_cyp 아 타장르 버닝중인데 현실생활이고 모고 내팽개쳐버리고 싶어서 죽겠다 크아악 덕질하는데 인생이 너무 방해댐 ㅡㅡ
@kim_yehwan RT @DanMi_02: ㅋㅋ 우리가 이렇게 말로 한남 패고 죽이고 이러는거 다 어디서 보고 듣고 경험해서 하고있는 거라고 생각해? 허구한날 여자는 삼일에 한번 패야한다 이딴 소리 짓껄이는 놈들이 인터넷에서 불특정 다수에 얻어걸려 말로 맞앗다고 꼴값…
@n4e74tTeGfwdDWN RT @M00M0W: 나이키 직원 불법촬영남. 집에서 120장이나 불법 촬영 사진이 발견됐는데, 회사 측에서는 피해자 분들한테 상품권 100만원 준다고 회유 시도

KeyboardInterrupt: 

### c5-05_streaming_api_with_tweepy 
- Tweepy 로 Streaming API 사용

In [38]:
import os
import tweepy

# 환경변수에서 인증 정보를 추출합니다.
# CONSUMER_KEY = os.environ['CONSUMER_KEY']
# CONSUMER_SECRET = os.environ['CONSUMER_SECRET']
# ACCESS_TOKEN = os.environ['ACCESS_TOKEN']
# ACCESS_TOKEN_SECRET = os.environ['ACCESS_TOKEN_SECRET']
CONSUMER_KEY        = "Rh9PPrGkDRohh5GualkQRErPu"
CONSUMER_SECRET     = "Emg0dTXJXawpWn4EapiyG2aE5bWHGLreFd6vPy4XZMQmVb7ULp"
ACCESS_TOKEN        = "4477564098-AGJFdwERnNyEqVGHaPyy8yyloiPaCXsPob1q1Zy"
ACCESS_TOKEN_SECRET = "otjvkl1U6POGZMw1SpGQHMHlyEPz9Z1TbhPIecFhCuI6e"

# 인증 정보를 설정합니다.
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

class MyStreamListener(tweepy.StreamListener):
    """
    Streaming API로 추출한 트윗을 처리하는 클래스입니다.
    """
    def on_status(self, status):
        """
        트윗을 받을 때 호출되는 메서드
        매개변수로 트윗을 나타내는 Status 객체가 전달됩니다.
        """
        print('@' + status.author.screen_name, status.text)

# 인증 정보와 StreamListener를 지정해서 Stream 객체를 추출합니다.
stream = tweepy.Stream(auth, MyStreamListener())

# 공개돼 있는 트윗을 샘플링한 스트림을 받습니다.
# 키워드 매개변수인 languages로 한국어 트윗만 추출합니다
stream.sample(languages=['ko'])
# stream.sample(languages=['en'])

@MD_pingnew @mxplaxit 따지자면 버그인게 맞으니 다시 고쳐주진 않겠죠..ㅠㅜ 스샷노기 안녕..
@dreamingu_mnks @72_Umezz 이 힝구에겐 당나귀귀 하셔도 무방합니다
@k_h_0602 🐻스타그램 

🐻「대」

#EXO #KAI #종인 #카이 https://t.co/05TtrIoRAX
@yutaisacutecat RT @jobdukeuni: 산이 미쳤나 ?? https://t.co/NXY6hqECZX
@KSH_JNF @Cake_JNF 아햐햐햨! 그래?? 화학화학!하고 웃는것보다는 이게 낫지않아? 특이한 웃음소리 듣고 구조하러 오면 좋겠다!!!그럼! 화학을 좋아하지 않으면 입주 불가능한 마을이라고! 완벽한 마을이지! ...?아니 조난은 재미있진 않지.
@1495beer 토요낮공 씅풍이잖아
@myforestO RT @theMEGASTUDY: 어서와 이렇게 영롱한 플래너는 처음이지? ٩( ᐛ ) و 

#비교불가 #반박불가 역대급 플래너가 왔댜 🎉

속지도 업그레이드! 메가쌤+대학로고 스티커까지!
안 갖고는 못 배기는 소. 장. 필. 템!🔥✨🤟🏻
갖고 싶지…
@_madestiny RT @0u0pp: Wanna One “1¹¹=1(POWER OF DESTINY)” 

1.Destiny (Intro.)
2.봄바람 *TITLE
3.집
4.불꽃놀이 (Flowerbomb)
5.묻고싶다 (One Love)
6.Deeper
7.술래 ht…
@HikaRi_lights @sulhwa_0808 ㄱㅋㅋㅋㄱㅋㅋㅋㅋㅋㄱㅋ 이게 말로만 듣던 재능낭비.?ㅋㅋㄱㅋ
@Eli_natsu @mybaekgimylife 진짜죠?? (울먹
@xoxohugnkiss RT @warm_garcon: ㅜㅜㅜ💛💛 https://t.co/BzrbFRTSfI
@Heen_foll 그런데 말입니다.왜 나랑 친해지려 하지 않을까요. 그것이궁금하다.
@1im_sujung 서울 인천 사시는 분들 중에 팝콘 콜라 꽁짜로 드셔주실분...?? 엄마한테 선물 주고 싶따요...ㅎ https://t.co/V

@asoka0 RT @BH_Hong2: 헉 진짜...칸예웨스트가 백현이 팔로했어...!!!🙊 https://t.co/FrwdW7Xkbw
@ttk0625 RT @anna57148315: 우리 입더러운 정마으문 신고 한번씩 해주세요 ~ https://t.co/UMmSoEa5mY
@Lin_a_bsd 아 히구치 귀여워ㅠㅠㅠㅠㅠ나말고도 히구치가 더 있던가 몬데ㅠㅠㅠㅠㅠㅠㅠㅠ선배를 좋아하면서도 선배가 자기를 좋아할거라 생각하지 못하는 우리 히구찌
@SriEndang_yani RT @StoneMusicEnt: 처음으로 보이는 워너원 멤버가 내 운명♥
@야, 내 운명 옹성우래~ 넌 누구나왔음??? 
　
[운명의 단어 찾기] Wanna One (워너원)
　
#WannaOne #워너원 #20181119_6PM
#POWEROFD…
@Mint_suga_o 포토티켓 어떻게 뽑아여??
@MeatMAC2 @WingSett 앗 완료 캡처 안 했는데ㅜ 암튼 했음~~ 팀플 힘내🤗💥💥
@penguiny22 RT @masigy: 같은 사람으로 안 보니까 그럴 수 있는거야.
사람이 아니라 당신보다 약한 '물건'이라고 생각하니까.
자기가 자기 인생 말아먹고나서야 후회한다? 그것도 똑똑한 사람들이야 알지 너넨 남탓하잖아.
심지어 피해자분들이 먼저 시비를 건…
@Blooming_Day_92 본격 수험생 없는 수능응원방송
@ayangmi1111 RT @Pianjue_: 아직도 자기가 새끼 때 몸인 줄 아는 대형견같아 너무귀여워ㅜㅠㅠㅠㅠㅠ https://t.co/3IpO44dKaX
@na_yoon_twt @CGVADGT1230 훟핳훟핳 기대가 됩미다
@jpsoo1565 RT @bhforever56: 칸예웨스트가 백현이 팔로했어 🙊🙊 https://t.co/yvegfmj31s
@dbwlsdlek_ RT @fangsongxo: 엑소, 美빌보드 메인차트 '아티스트100' 9위+TOP10 첫 진입[공식입장] | 다음연예 https://t.co/6avHgQUP7L
@thfthf6972 RT @712_ATM: 케모노

KeyboardInterrupt: 